# UAS Flight Analysis: ClearSky

## Setup

### Imports

In [ ]:
%load_ext autoreload
%autoreload 2

import json
import os
import sys

from datetime import datetime, timedelta

import hvplot
import hvplot.pandas
import hvplot.xarray
import numpy as np
import pandas as pd
import xarray as xr

import holoviews as hv
from bokeh.models.renderers import GlyphRenderer
from bokeh.models import Range1d, LinearAxis, DatetimeTickFormatter

hv.extension('bokeh')

# sys.path.insert(0, "/home/derek/Software/python/envDataSystem_analysis/utilities")
flight_path = os.path.abspath("")
proj_path = os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath(""))))
code_path = os.path.join(proj_path, "code")
if proj_path not in sys.path:
    sys.path.insert(0, proj_path)
if code_path not in sys.path:
    sys.path.insert(0, code_path)
# print(sys.path)
from convert import Data, dt_to_string, string_to_dt
from dataevent import (
    DataEvent,
    load_inverted_msems,
    process_pops,
    process_payload,
    process_piccolo,
    save_flight_itx
)

### Data systems
Define the datasystem(s). This includes base paths, instruments with variable maps and extra parameters specific to each instrument. This should change to much flight to flight but the paths might need to be changed to point to the data.

In [ ]:
# get settings.json
with open(os.path.join("config", "settings.json")) as f:
    config = json.load(f)

flight_datasystems = {
    "ClearSky": {
        "payload": {
            # "base_path": "/path/to/your/data/to/override",
            "base_path": "./data/payload",
            "instruments": {
                "payload": {
                    "format": "clear-payload-dat",
                    "timebase": 1,
                    "dims": ["time"],
                    "variables": [
                        ("CONCN", "CONCN", "CN concentration"),  # ,#/cc
                        ("ABSRA", "ABSRA", "PSAP red sample"),  # counts,
                        ("ABSRB", "ABSRB", "PSAP red reference"),  # counts,
                        ("ABSGA", "ABSGA", "PSAP green sample"),  # counts,
                        ("ABSGB", "ABSGB", "PSAP green reference"),  # counts,
                        ("ABSBA", "ABSBA", "PSAP blue sample"),  # counts,
                        ("ABSBB", "ABSBB", "PSAP blue reference"),  # counts,
                        ("CHMPS", "CHMPS", "Chem filter number"),  # number,
                        ("AT", "AT", "Air temp from slow probe"),  # ,Deg C
                        ("RH", "RH", "RH from slow probe"),  # ,%
                        ("SMPFL", "SMPFL", "MCPC sample flow"),  # ,cc/min
                        (
                            "SMPFP",
                            "SMPFP",
                            "MCPC sample pump power settings",
                        ),  # (range 0 – 200),
                        ("SATFL", "SATFL", "MCPC Saturator flow"),  # ,cc/min
                        (
                            "SATFP",
                            "SATFP",
                            "MCPC saturator pump power settings",
                        ),  # (range 0 – 200),
                        ("ABSFL", "ABSFL", "PSAP flow"),  # ,cc/min
                        (
                            "ABSFP",
                            "ABSFP",
                            "MCPC saturator pump power settings",
                        ),  # (range 0 – 200),
                        ("CHMFL", "CHMFL", "Chem filter flow"),  # ,cc/min
                        (
                            "CHMFP",
                            "CHMFP",
                            "Chem filter pump power settings",
                        ),  # ( 0 – 200),
                        ("OPCFL", "POPS_FL", "POPS flow"),  # ,cc/min
                        ("OPCFP", "OPCFP", "POPS pump power settings"),  # (0 – 200),
                        ("OPTCT", "OPTCT", "MCPC optics block temp"),  # ,deg C
                        ("OPTCP", "OPTCP", "MCPC optics block power"),  # ,
                        ("CONDT", "CONDT", "MCPC condenser temperature"),  # ,deg C
                        ("CONDP", "CONDP", "MCPC condenser power settings"),  # ,
                        ("SATTT", "SATTT", "MCPC Saturator top temp"),  # ,deg C
                        ("SATTP", "SATTP", "MCPC Saturator power setting"),  # ,
                        ("SATBT", "SATBT", "MCPC Saturator bottom temp"),  # ,deg C
                        ("SATBP", "SATBP", "MCPC Saurator bottom power setting"),  # ,
                        ("INLTT", "INLTT", "Temp at the LEF manifold"),  # ,deg C
                        (
                            "FILLC",
                            "FILLC",
                            "BuOH indicator, starts incrementing when BuOH is below full",
                        ),  # ,
                        (
                            "CABNT",
                            "CABNT",
                            "Temp on the MCPC side of payload",
                        ),  # ,deg C
                        (
                            "PRESS",
                            "PRESS",
                            "Absolute pressure at in the flow manifold",
                        ),  # ,mb
                        ("PSAP-T", "PSAP_T", "PSAP Temp"),  # ,deg C
                        ("PSAP-RH", "PSAP-RH", "PSAP RH"),  # ,%
                        ("POPS-T", "POPS_T", "POPS Temp"),  # ,deg C
                        ("POPS-RH", "POPS_RH", "POPS RH"),  # ,%
                        ("FastT", "FastT", "Fast temp sensor"),  # ,deg C
                        ("FastRH", "FastRH", "Fast RH sensor"),  # ,%
                    ],
                },
            },
        },
        "pops": {
            # "base_path": "/path/to/your/data/to/override",
            "base_path": "./data/pops",
            "instruments": {
                "pops": {
                    "format": "pops-bin",
                    "timebase": 30,
                    "resample": False,
                    "dims": ["time", "pops_bins"],
                    "bin_count": 26,
                    "calibration_file": "/home/derek/Data/UAS/pops_calibration/Cal_curve.csv",
                    "process_options": {"type": "pops"},
                    "variables": [
                        ("bin_counts", "pops_bin_counts", "clear_pops_bin_counts"),
                        ("diameter_um", "pops_dp_um_2d", "cloudy_msems_diameter_um"),
                    ],
                }
            },
        },
    },
    "GroundStation": {
        "uasground": {
            # "base_path": "/path/to/your/data/to/override",
            "base_path": config["data_paths"]["envdsys_groundstation_data_path"],
            "instruments": {
                "amcpc_cn": {
                    "format": "envdsys",
                    "timebase": 1,
                    "dims": ["time"],
                    "variables": [
                        ("concentration", "ground_cn", "amcpc_cn_concentration"),
                    ],
                },
                # "magic_cn": {
                #     "format": "envdsys",
                #     "timebase": 1,
                #     "dims": ["time"],
                #     "variables": [
                #         ("concentration", "magci_cn", "magic_cn_concentration"),
                #     ]
                # }
            },
        }
    },
    "AutoPilot": {
        "navigation": {
            # "base_path": "/alternate/path/to/override", 
            "base_path": "./data/piccolo",
            "instruments": {
                "piccolo": {
                    "format": "piccolo-log",
                    "timebase": 1,
                    "dims": ["time"],
                    "variables": [
                        ("Lat", "latitude", ""),
                        ("Lon", "longitude", ""),
                        ("Height", "altitude", ""),
                        ("GroundSpeed", "ground_speed", ""),
                        ("Direction", "heading", ""),
                        ("BaroAlt", "pressure_altitude", ""),
                        ("TAS", "true_air_speed", ""),
                        ("Roll", "roll", ""),
                        ("Pitch", "pitch", ""),
                        ("Yaw", "yaw", ""),
                        ("MagHdg", "heading_mag", ""),
                        ("AGL", "height_agl", ""),
                    ]
                }
            }
        }
    }
}

## Flight Details

### Flight ID

In [ ]:
# override settings here
# project = "<ProjectID>"
# # project_long_name = "<Long Project Name>"
# platform = "<PlatformName>" # FVR-55, AeroPhys, RHBrown
# flight_id = "<Flight_??>"

project = config["project"]
# project_long_name = "<Long Project Name>"
platform = config["platform"] # FVR-55, AeroPhys, RHBrown
flight_id = config["flight_id"]

### Event Data

In [ ]:
# shouldn't change this if using the templates
# payload_id = "ClearSky"
payload_id = config["payload_id"]

flight_event_config = {
    "preflight": {},
    "flight": {
        "start_time": "YYYY-mm-ddTHH:MM:SSZ", # enter valid start time
        "end_time": "YYYY-mm-ddTHH:MM:SSZ",   # enter valid start time
        "datasystems": ["ClearSky", "AutoPilot", "GroundStation"],
    },
    "postflight": {},
}

# --- automatically generated - don't edit below this line ---
flight_config = {
    "kind": "ClearSkyFlight",
    "metadata": {"project": project, "platform": platform, "flight_id": flight_id, "payload_id": payload_id},
    "events": flight_event_config,
    "datasystems": flight_datasystems,
}
# flight_config

## Load Data
Un/comment lines as needed. Multiple events can be processed if wanted. Will load data based on Event Data cell above

### Create Event(s)

In [ ]:
# create preflight event
# preflight_event = DataEvent("preflight", config=flight_config)

# create flight event
flight_event = DataEvent("flight", config=flight_config)

# create postflight event
# postflight_event = DataEvent("postflight", config=flight_config)

### Process data

#### Payload

In [ ]:
# pre_msems = preflight_event.get_dataset("cloudy_cdp", datasystem="CloudySky", controller="uas_cloudy")
# process_msems(pre_msems)
payload = flight_event.get_dataset("payload", datasystem="ClearSky", controller="payload")
ref_init = {
    # payload.time.values[0]: {
    '2022-02-16T20:00:00': {
        "init_450": 0.813,
        "init_525": 0.813,
        "init_624": 0.819,
    },
}

flight_event.process_payload("payload", datasystem="ClearSky", controller="payload", ref_init=ref_init)
payload = flight_event.get_dataset("payload", datasystem="ClearSky", controller="payload")
# process_cdp(cdp)

# post_msems = flight_event.get_dataset("cloudy_cdp", datasystem="CloudySky", controller="uas_cloudy")
# process_msems(post_msems)
# payload

#### POPS

In [ ]:
# preflight_event.process_pops("pops", datasystem="ClearSky", controller="pops")
# preflight_pops = flight_event.get_dataset("pops", datasystem="ClearSky", controller="pops")

flight_event.process_pops("pops", datasystem="ClearSky", controller="pops")
pops = flight_event.get_dataset("pops", datasystem="ClearSky", controller="pops")
# process_cdp(cdp)

# postflight_event.process_pops("pops", datasystem="ClearSky", controller="pops")
# postflight_pops = flight_event.get_dataset("pops", datasystem="ClearSky", controller="pops")
# pops

### Merge datasets
Combine datasets on a common timebase. User can specify timebase (tb=1 is default) and list of datasets (ds_list). If ds_list is omitted, all available datasets are merged.

In [ ]:
# preflight = preflight_event.merge()
# preflight_30s = preflight_event.merge(tb=30, ds_list=["pops", "payload", "amcpc_cn"])
# if "POPS_FL" in preflight_30s:
#     preflight_30s = process_pops(preflight_30s, flow_rate=(preflight_30s.POPS_FL / 60))

flight = flight_event.merge()
flight_30s = flight_event.merge(tb=30, ds_list=["pops", "payload", "amcpc_cn"])
if "POPS_FL" in flight_30s:
    flight_30s = process_pops(flight_30s, flow_rate=(flight_30s.POPS_FL / 60))

# postflight = postflight_event.merge()
# postflight_30s = postflight_event.merge(tb=30, ds_list=["pops", "payload", "amcpc_cn"])
# if "POPS_FL" in postflight_30s:
#     postflight_30s = process_pops(postflight_30s, flow_rate=(postflight_30s.POPS_FL / 60))

### Save datasets

In [ ]:
# save_flight_itx(preflight)
# save_flight_itx(preflight_30s)

save_flight_itx(flight)
save_flight_itx(flight_30s)

# save_flight_itx(postflight)
# save_flight_itx(postflight_30s)

## Data Analysis
Add descriptive text in [markdown](https://jupyter-notebook.readthedocs.io/en/latest/examples/Notebook/Working%20With%20Markdown%20Cells.html) cells and run code in the default code cells. 

*This would be description of particular data analysis*

In [ ]:
# code cell
flight

In [ ]:
flight_30s

### Visualization

#### Standard Plot Definitions
This is a list of standard plots that we might want to see. Run the following cell to create the plots for use below. 

In [ ]:
dt_formatter = DatetimeTickFormatter(
    years="%Y", months="%Y-%m", days="%F", hours="%m-%d %H:%M", hourmin="%m-%d %H:%M", minutes="%H:%M", minsec="%T", seconds="%T", milliseconds="%T.%1N"
)

pops_dndlogdp_im = (
    flight_30s.set_coords("time_mid")
    .swap_dims({"time": "time_mid", "pops_bins": "pops_dp_um"})
    .reset_coords()
    .pops_dNdlogDp.hvplot.quadmesh(
        x="time_mid",
        y="pops_dp_um",
        logy=True,
        cmap="rainbow",
        responsive=True,
        min_width=300,
        min_height=300,
        xformatter=dt_formatter,
        label="POPS dNdlogDp"
    )
)
pops_dndlogdp_scans = (
    flight_30s.set_coords("time_mid")
    .swap_dims({"pops_bins": "pops_dp_um"})
    .reset_coords()
    .pops_dNdlogDp.hvplot.line(
        x="pops_dp_um", logx=True, responsive=True, min_width=300, min_height=300, label="POPS dNdlogDp"
    )
)

pops_dsdlogdp_im = (
    flight_30s.set_coords("time_mid")
    .swap_dims({"time": "time_mid", "pops_bins": "pops_dp_um"})
    .reset_coords()
    .pops_dSdlogDp.hvplot.quadmesh(
        x="time_mid",
        y="pops_dp_um",
        logy=True,
        cmap="rainbow",
        responsive=True,
        min_width=300,
        min_height=300,
        xformatter=dt_formatter,
        label="POPS dSdlogDp"
    )
)
pops_dsdlogdp_scans = (
    flight_30s.set_coords("time_mid")
    .swap_dims({"pops_bins": "pops_dp_um"})
    .reset_coords()
    .pops_dSdlogDp.hvplot.line(
        x="pops_dp_um", logx=True, responsive=True, min_width=300, min_height=300, label="POPS dSdlogDp"
    )
)

pops_dvdlogdp_im = (
    flight_30s.set_coords("time_mid")
    .swap_dims({"time": "time_mid", "pops_bins": "pops_dp_um"})
    .reset_coords()
    .pops_dVdlogDp.hvplot.quadmesh(
        x="time_mid",
        y="pops_dp_um",
        logy=True,
        cmap="rainbow",
        responsive=True,
        min_width=300,
        min_height=300,
        xformatter=dt_formatter,
        label="POPS dVdlogDp"
    )
)
pops_dvdlogdp_scans = (
    flight_30s.set_coords("time_mid")
    .swap_dims({"pops_bins": "pops_dp_um"})
    .reset_coords()
    .pops_dVdlogDp.hvplot.line(
        x="pops_dp_um", logx=True, responsive=True, min_width=300, min_height=300, label="POPS dVdlogDp"
    )
)

pops_intN_ts = flight_30s.pops_intN.hvplot.scatter(
    label="POPS intN", responsive=True, min_width=300, min_height=300, xformatter=dt_formatter
)
pops_intS_ts = flight_30s.pops_intS.hvplot.scatter(
    label="POPS intS", responsive=True, min_width=300, min_height=300, xformatter=dt_formatter
)
pops_intV_ts = flight_30s.pops_intV.hvplot.scatter(
    label="POPS intV", responsive=True, min_width=300, min_height=300, xformatter=dt_formatter
)

payload_cn_ts = flight.CONCN.hvplot.scatter(
    label="CN", responsive=True, min_width=300, min_height=300, xformatter=dt_formatter
)

bap_450_ts = flight.bap_450.hvplot.scatter(c="blue", label="Bap(450nm)", responsive=True, min_width=300, min_height=300, xformatter=dt_formatter)
bap_525_ts = flight.bap_525.hvplot.scatter(c="green", label="Bap(525nm)", responsive=True, min_width=300, min_height=300, xformatter=dt_formatter)
bap_624_ts = flight.bap_624.hvplot.scatter(c="red", label="Bap(624nm)", responsive=True, min_width=300, min_height=300, xformatter=dt_formatter)

if "altitude" in flight_30s:
    pops_intN_alt = (
        flight_30s.set_coords("pops_intN")
        .swap_dims({"time": "pops_intN"})
        .reset_coords()[["altitude"]]
        .hvplot.scatter(
            x="pops_intN",
            y="altitude",
            label="POPS intN",
            responsive=True,
            min_width=300,
            min_height=300,
            xformatter=dt_formatter
        )
    )
    pops_intS_alt = (
        flight_30s.set_coords("pops_intN")
        .swap_dims({"time": "pops_intS"})
        .reset_coords()[["altitude"]]
        .hvplot.scatter(
            x="pops_intS",
            y="altitude",
            label="POPS intS",
            responsive=True,
            min_width=300,
            min_height=300,
            xformatter=dt_formatter
        )
    )
    pops_intV_alt = (
        flight_30s.set_coords("pops_intN")
        .swap_dims({"time": "pops_intV"})
        .reset_coords()[["altitude"]]
        .hvplot.scatter(
            x="pops_intV",
            y="altitude",
            label="POPS intV",
            responsive=True,
            min_width=300,
            min_height=300,
            xformatter=dt_formatter
        )
    )
    payload_cn_alt = (
        flight.set_coords("CONCN")
        .swap_dims({"time": "CONCN"})
        .reset_coords()[["altitude"]]
        .hvplot.scatter(
            x="CONCN",
            y="altitude",
            # c="ambient_rh",
            label="cn",
            # cmap="rainbow",
            responsive=True,
            min_width=300,
            min_height=300,
            xformatter=dt_formatter
        )
    )

    bap_450_alt = (
            flight.set_coords("bap_450")
            .swap_dims({"time": "bap_450"})
            .reset_coords()[["altitude"]]
            .hvplot.scatter(
                x="bap_450",
                y="altitude",
                c="blue",
                label="Bap(450nm)",
                responsive=True,
                min_width=300,
                min_height=300,
                xformatter=dt_formatter
            )
        )

if "ground_cn" in flight:
    ground_cn_ts = flight.ground_cn.hvplot.scatter(
        label="ground cn", responsive=True, min_width=300, min_height=300, xformatter=dt_formatter
    )

#### Data Plots
Compose plots by using the assigned name above (or create new ones). Use "+" and "\*" to create plots side-by-side or overlayed, respectively. E.g., 

```msems_intN + msems_intS``` 

will display plots of intN and intS side by side and will share the time axis.

In [ ]:
# Example compositions
# msems_dndlogdp_im + msems_dndlogdp_scans
# msems_dndlogdp_im + msems_dsdlogdp_im + msems_dvdlogdp_im
# msems_intN+msems_intS+msems_intV
(bap_450_ts*bap_525_ts*bap_624_ts).opts(ylim=(0,2)) + ground_cn_ts.opts(ylim=(0,2000))